In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd


/home/gk/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# ReLU

Rectified Linear Unit (ReLU) is a goto activation function for hidden layers in present days replacing sigmoid function which was generally. It replaces negative activations with zero


In [2]:
def ReLU(Z_values):
    return np.maximum(0,Z_values)

In [3]:

def MLP():
    pass
    

In [4]:
def forward_propagation(X, Y, Layers, Weights=[], Biases=[]):
    # Setting weights and biases for random values if unspecified
    if len(Weights) == 0:
        Weights.append(np.random.rand(X.shape[1],Layers[0]))
        for i in range(1,len(Layers)):
            Weights.append(np.random.rand(Weights[i-1].shape[1],Layers[i]))
    if len(Biases) == 0:
        Biases = [np.random.rand(n_units) for n_units in Layers]
    
    Z_values = []
    Activations = []
#     for i in Weights:
#         print(i.shape)
    # Setting up the 1st layer
    dot_product = np.dot(X, Weights[0]) 
    L1_value = dot_product + Biases[0][np.newaxis, :]
    Z_values.append(L1_value)
    Activations.append(ReLU(L1_value))  # Using L1_value here
    
    # Iterating for the remaining layers
    for i in range(len(Layers)-1):  
        dot_product = np.dot(Activations[i], Weights[i+1]) # Start from 1, since the first layer is already handled
        Z_value=dot_product+ Biases[i+1]
        Z_values.append(Z_value)
        Activations.append(ReLU(Z_value))

    
    return  Activations, Weights, Biases,Z_values,


In [5]:
value,_,_,_=forward_propagation(np.random.rand(10,3),[1,2,3,4,5,6,7,8,9,10],[4,2,1])
for i in value:
    print(f"{len(i)} \t {i} \t Shape:{i.shape} \n")

10 	 [[1.06920067 2.104715   1.55729685 1.85038308]
 [0.94095158 1.91144357 1.55090212 1.93308128]
 [0.90579863 1.54458702 1.13694114 1.51443638]
 [0.57012035 1.68949169 0.75868088 1.2674275 ]
 [0.61058223 1.65121325 1.30467679 1.86609261]
 [0.70111859 1.28620763 1.14530171 1.66333103]
 [0.81576991 1.75855305 0.93369633 1.31836098]
 [1.01736137 1.82775441 1.46247471 1.79567234]
 [0.8740164  1.36021535 1.10686242 1.51389556]
 [0.77499237 1.79739762 1.2142509  1.65667514]] 	 Shape:(10, 4) 

10 	 [[3.74292078 3.64223857]
 [3.56905214 3.68306048]
 [2.89890235 2.85573077]
 [2.70881391 2.26562605]
 [3.13550629 3.33798821]
 [2.68092213 2.96014108]
 [2.9154098  2.50088312]
 [3.42073284 3.46576745]
 [2.71227371 2.80311712]
 [3.18627167 3.08393476]] 	 Shape:(10, 2) 

10 	 [[4.69392806]
 [4.71491863]
 [3.81938592]
 [3.2129075 ]
 [4.32413309]
 [3.89848843]
 [3.46935593]
 [4.48281661]
 [3.74629189]
 [4.07789463]] 	 Shape:(10, 1) 



In [6]:
def calculate_cost(Y,calculated_value):
    difference=(np.array(Y)-calculated_value)**2
    return np.sum(difference)/len(Y)
    
    

In [7]:
def calculate_gradients(Weights,Biases,d_cost_by_activation,d_activation_by_Z):    
   
    d_Z_by_weight=Weights
    d_cost_by_weight=d_cost_by_activation * d_activation_by_Z*d_Z_by_weight
    d_Z_by_bias=1 
    d_cost_by_bias=np.sum(d_cost_by_activation * d_activation_by_Z*d_Z_by_bias)
    return d_cost_by_weight,d_cost_by_bias

In [8]:


def backward_propagation(X,Y,W,b,Activations,learning_rate):
    d_cost_by_activation=np.sum(Activations[-1]-np.array(Y).reshape(-1, 1))
#     print(d_cost_by_activation,'\n',Activations[-1],'\n',np.array(Y))
    d_activation_by_Z=1
    new_activation=Activations.insert(0,X)
    new_weights=[]
    new_biases=[]
    for i in range(len(W)-1,0,-1):
        calc_weight,calc_bias=calculate_gradients(new_activation,b[i],d_cost_by_activation,d_activation_by_Z)
        print(calc_weight,"W[i]\n",W[i])
        n_layer_wt=W[i]-learning_rate*calc_weight[i]
        n_layer_b=b[i]-learning_rate*calc_bias[i]
        new_weights.append(n_layer_wt)
        new_biases.append(n_layer_b)
    return new_weights,new_biases


In [9]:

def MLP(X,y,Layers,iterations,learning_rate,error_margin):
    iteration_count=0
    cost=100
    cost_history=[]
    W=[]
    B=[]
    while iteration_count<iterations or cost<=error_margin :
        print(f"Iteration: {iteration_count+1}")
        activation,weight,bias,_=forward_propagation(X,y,Layers,W,B)
        cost=calculate_cost(y,activation[-1])
        cost_history.append(cost)
        W,B=backward_propagation(X,y,weight,bias,activation,learning_rate)
        iteration_count+=1
    return weights,biases,layers,cost_history

In [10]:
wt,bias,layers,cost_history=MLP(np.random.rand(10,3),[1,2,3,4,5,6,7,8,9,10],[4,2,1],100,0.05,0.5)


Iteration: 1


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
aa=[1,2,3]
b=[4,5,6]

np.array(b)-np.array(aa)

In [ ]:
np.random.rand?

In [ ]:
a={1:"a",2:"B"}
a.values()

In [ ]:
def func1():
    def func2():
        print("konichiwas broskis")


In [ ]:
func1()

In [ ]:
aa.insert(0,5)
aa[-1]

In [ ]:
a=[1,2,3,4,5]
a[1:]